# Testing cross project artifacts

In [ ]:
!pip install -U xgboost
!pip install scikit-learn

## 1. artifact generating function 

In [ ]:
#mlrun: start-code

import mlrun
import pandas as pd 
import json
import os
from xgboost import XGBClassifier
import pickle
from mlrun.artifacts.base import DirArtifact
from mlrun import MLClientCtx
from sklearn.datasets import load_iris
from io import BytesIO
from sklearn.model_selection import train_test_split

def get_dataitem(context: MLClientCtx,
                             key: str):
    
    for artifact in context.artifacts:
        if artifact['kind'] == 'model' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'] + artifact['spec']['model_file'])
        elif artifact['kind'] == 'dataset' and artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
        elif artifact['metadata'].get('key',None) == key:
            return mlrun.get_dataitem(artifact['spec']['target_path'])
    context.logger.info('Artifact not found')
    
def log_transactions(context: MLClientCtx,
                    ):
        
    # uploading new artifact 
    df_encode = pd.DataFrame(load_iris()['data']).to_json().encode()
    context.log_artifact('encoded_iris-'+context.artifact_path[:2], body=df_encode, local_path='encoded_iris-'+context.artifact_path[:2]+'.csv')
    # reading artifact
    trans_df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get()))
    context.logger.info(f'dataframe shape : {trans_df.shape}')
    
    # training the model (for serving purposes )
    bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
    X,y = load_iris(return_X_y=True)
    X_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
    bst.fit(X_train, y_train)
    # logging a model
    context.log_model('bst_model', body=pickle.dumps(bst), model_file='bst.pkl')
    # getting the model remote
    model = pickle.loads(get_dataitem(context, 'bst_model').get())
    context.logger.info(f'logged model : {model.__class__}')
    
    # Logging directory
    context.log_artifact(DirArtifact(key='my_project', target_path=context.artifact_path))
    
    # Logging dataset 
    context.log_dataset(key = 'iris_dataset-'+context.artifact_path[:2],
                        df = pd.DataFrame(json.loads(get_dataitem(context, 'encoded_iris-'+context.artifact_path[:2]).get())),
                        local_path='iris_dataset-'+context.artifact_path[:2]+'.csv')
    # Getting dataset
    context.logger.info(f'logged dataset {get_dataitem(context, "iris_dataset-" + context.artifact_path[:2]).as_df().shape}')
    
    return
            
#mlrun: end-code

## 2. Creating projects, setting & running artifact generating function

In [ ]:
import mlrun
import os

# Initialize the MLRun project object
project1 = mlrun.get_or_create_project('cross-project1',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks1'))

func = project1.set_function(name='log_transactions', kind='job', image='mlrun/mlrun', handler='log_transactions', requirements=["xgboost"])
func.deploy()
project1.get_function('log_transactions').run(local=False)

In [ ]:
# Initialize the MLRun project object
project2 = mlrun.get_or_create_project('cross-project2',user_project=True,context=os.path.join(os.getcwd(), 'test-notebooks2'))

func = project2.set_function(name='log_transactions', kind='job', image='mlrun/mlrun', handler='log_transactions', requirements=["xgboost"])
func.deploy()
project2.get_function('log_transactions').run(local=False)

## 3. Importing/Exporting artifacts

### 3.1 base artifact

In [ ]:
# Exporting project2 artifact
project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').export('proj2_encoded_iris.yaml')

# Importing the artifact from project1
project1.import_artifact('../proj2_encoded_iris.yaml',
                         new_key = 'imported_proj1_encoded_iris')

# Testing the imported artifact
pd.DataFrame(json.loads(project1.get_artifact('imported_proj1_encoded_iris').to_dataitem().get())).head()

### 3.2 dataset artifact

In [ ]:
# Exporting project2 model artifact
project2.get_artifact('log-transactions-log-transactions_iris_dataset-v3').export('proj2_dataset.yaml')

# importing model artifact from project2
project1.import_artifact('../proj2_dataset.yaml',
                         new_key = 'imported_proj1_dataset')

# Testing the imported dataset artifact
project1.get_artifact('imported_proj1_dataset').to_dataitem().as_df()

### 3.3 model artifact

In [ ]:
# Exporting project1 model artifact
project1.get_artifact('log-transactions-log-transactions_bst_model').export('proj1_model.yaml')

# importing model artifact from project2
project2.import_artifact('../proj1_model.yaml',
                         new_key = 'imported_proj2_model')


# Testing the imported model artifact
project2_model = pickle.loads(project2.get_artifact('imported_proj2_model')._get_file_body())
project2_model.predict(pd.DataFrame(json.loads((project1.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())))[:5])

### 3.4 dir artifact

In [ ]:
try: 
    # Exporting project2 dir artifact
    project1.get_artifact('log-transactions-log-transactions_my_project').export('proj1_dir.yaml')

    # importing dir artifact from project2
    project2.import_artifact('../proj1_dir.yaml',
                             new_key = 'imported_proj2_dir')

    # Testing the imported dir artifact
    print(project2.get_artifact('imported_proj2_dir').to_dataitem().listdir())
except Exception as e:
    print(e)

## 4. Importing/Exporting functions

### 4.1 Serving function

#### 4.1.1 Creating serving function, adding model, predicting and deploying

In [ ]:
# Setting project1 serving function
func = project1.set_function(name='proj1-serving',func = mlrun.new_function(name='proj1-serving', kind='serving',image='mlrun/mlrun', command = [], requirements=["xgboost"]))
# adding project2 model to project1 serving function
project1.get_function('proj1-serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project2.get_artifact('log-transactions-log-transactions_bst_model').target_path)

In [ ]:
# Testing the fused model serving function
server = project1.get_function('proj1-serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

In [ ]:
project1.deploy_function('proj1-serving')

In [ ]:
import time
time.sleep(5)

project1.get_function('proj1-serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

#### 4.1.2 Importing & Exporting already deployed serving function

In [ ]:
# exporting project1 deployed serving function
project1.get_function('proj1-serving').export('proj1-serving.yaml')

# importing project1 serving function from project2
project2.set_function(name='proj2-imported-serving', func='../proj1-serving.yaml')
print(project2.get_function('proj2-imported-serving').is_deployed())

In [ ]:
try:
    # Testing the imported already deployed function
    server = project2.get_function('proj2-imported-serving').to_mock_server()

    server.test(body={'inputs': 
                      pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})
except Exception as e:
    print(e)

In [ ]:
# Redeploying the already deployed function
project2.deploy_function('proj2-imported-serving')

##### Testing the redeployed imported-deployed-function

In [ ]:
time.sleep(5) 
project2.get_function('proj2-imported-serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

#### 4.1.3 Importing & Exporting undeployed serving function

In [ ]:
# Setting project2 serving function
func = project2.set_function(mlrun.new_function(name='proj2-serving', kind='serving',image='mlrun/mlrun', command = [], requirements=["xgboost"]))
# adding project1 model to project2 serving function
project2.get_function('proj2-serving').add_model(key = 'my_model',
                                                 class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                                 model_path = project1.get_artifact('log-transactions-log-transactions_bst_model').target_path)

In [ ]:
# exporting project2 not deployed serving function
project2.get_function('proj2-serving').export('proj2-serving.yaml')

# Importing project2 serving function from project1
project1.set_function(name='proj1-imported-serving', func='../proj2-serving.yaml')
print(project1.get_function('proj1-imported-serving').is_deployed())

In [ ]:
# Testing the imported undeployed function
server = project1.get_function('proj1-imported-serving').to_mock_server()

server.test(body={'inputs': 
                  pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]})

In [ ]:
# Deploying the undeployed imported function
project1.deploy_function('proj1-imported-serving')

In [ ]:
time.sleep(5)

# Testing the imported then deployed function
project1.get_function('proj1-imported-serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': pd.DataFrame(json.loads(project2.get_artifact('log-transactions-log-transactions_encoded_iris-v3').to_dataitem().get())).values.tolist()[-5:]}
)

### 4.2 mlrun function

In [ ]:
# exporting project2 log_transaction mlrun function
project2.get_function('log-transactions').export('proj2-mlrun-func.yaml')

# Importing project2 log_transaction mlrun function from project1
project1.set_function(name='proj1-imported-mlrun-func', func='../proj2-mlrun-func.yaml')
project1.run_function('proj1-imported-mlrun-func')

### 4.3 nuclio function

#### 4.3.1 Importing & Exporting deployed remote function

In [ ]:
%%writefile script.py
import mlrun
import time

def handler(context: mlrun.MLClientCtx, event):
    context.logger.info('Going to sleep zZz...')
    time.sleep(5)
    return 'Waking up !'

In [ ]:
# Setting remote function
project1.set_function(func = mlrun.code_to_function(name='proj1-remote-func', kind='remote',image='mlrun/mlrun', filename='script.py', handler='handler', requirements=["xgboost"]))

# Deploying
project1.deploy_function('proj1-remote-func')
# Exporting project1 undeployed remote function
project1.get_function('proj1-remote-func').export('proj1-remote-func.yaml')

# Importing project2 remote function from project1
project2.set_function(name='proj2-imported-remote-func', func='../proj1-remote-func.yaml')

In [ ]:
project2.deploy_function('proj2-imported-remote-func')

time.sleep(5)
project2.get_function('proj2-imported-remote-func').invoke('')

## 5. Importing/Exporting project

In [ ]:
# Initialize the MLRun project object
new_project = mlrun.get_or_create_project('testing-exported-proj', context=os.path.join(os.getcwd(), 'new-project'))

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS, S3_BUCKET
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID', None)
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY', None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None)
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None

secrets = {'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
           'AWS_SECRET_ACCESS_KEY':AWS_SECRET_ACCESS_KEY,
           'GOOGLE_APPLICATION_CREDENTIALS': GOOGLE_APPLICATION_CREDENTIALS}

new_project.set_secrets(secrets=secrets, provider='kubernetes')

S3_BUCKET = os.environ.get('S3_BUCKET', 'testbucket-igz-temp')
GS_BUCKET = os.environ.get('GS_BUCKET', 'testbucket-igz')

new_project.artifact_path = os.path.join('s3://', S3_BUCKET + '/cross_project/')

In [ ]:
# Setting artifacts
new_project.set_artifact('encoded_iris', artifact=os.path.join(os.getcwd(),'proj2_encoded_iris.yaml'))
new_project.set_artifact('model', artifact=os.path.join(os.getcwd(),'proj1_model.yaml'))
new_project.set_artifact('dataset', artifact=os.path.join(os.getcwd(),'proj2_dataset.yaml'))
new_project.set_artifact('dir', artifact=os.path.join(os.getcwd(),'proj1_dir.yaml'))

# Setting functions
new_project.set_function(name='serving', func='../proj1-serving.yaml')
new_project.set_function(name='mlrun-func', func='../proj2-mlrun-func.yaml')
new_project.set_function(name='remote-func', func='../proj1-remote-func.yaml')


In [ ]:
# exporting the project

# S3 artifact path (e.g. s3://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join(new_project.artifact_path, 'new_project.zip'))

# GCS artifact path (e.g. gs://my-bucket/new_project.zip)
new_project.export(filepath=os.path.join('gs://' + GS_BUCKET, 'new_project.zip'))

# V3IO local path
new_project.export(filepath='/v3io/bigdata/new_project.zip')

In [ ]:
# Importing the projects
gs_project = mlrun.load_project(name='testing-gs-project',
                                url=os.path.join(new_project.artifact_path, 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 'gs-project'))

s3_project = mlrun.load_project(name='testing-s3-project',
                                url=os.path.join('s3' + new_project.artifact_path[2:], 'new_project.zip'),
                                context = os.path.join(os.getcwd(), 's3-project'))

v3io_project = mlrun.load_project(name='testing-v3io-project',
                                  url='/v3io/bigdata/new_project.zip',
                                  context=os.path.join(os.getcwd(), 'v3io-project'))

In [ ]:
# Testing the imported artifacts
# Artifact
gs_encoded_dataset = pd.DataFrame(json.loads(gs_project.get_artifact('encoded_iris').to_dataitem().get()))
print('gs project artifact\n', gs_encoded_dataset.head(), '\n\n')

# Dataset
s3_dataset = s3_project.get_artifact('dataset').to_dataitem().as_df()
print('s3 project artifact\n',s3_dataset.head())

# Model
v3io_model = pickle.loads(v3io_project.get_artifact('model')._get_file_body())
v3io_model.predict(gs_encoded_dataset[:5])

# Directory
gs_dir = gs_project.get_artifact('dir').to_dataitem().listdir()

# Testing imported functions
# Serving function
gs_project.get_function('serving').deploy()
time.sleep(5)
gs_project.get_function('serving').invoke(
    path='/v2/models/my_model/infer', 
    body={'inputs': gs_encoded_dataset.values[-5:].tolist()}
)

# mlrun function
s3_project.run_function('mlrun-func',local=True)

## nuclio function
v3io_project.deploy_function('remote-func')

time.sleep(5)
v3io_project.get_function('remote-func').invoke('')

## Cleanup

In [ ]:
projects = [project1,project2,gs_project,s3_project,v3io_project, new_project]
for project in projects:
    mlrun.get_run_db().delete_project(name=project.name, deletion_strategy='cascade')

import shutil
for f in os.listdir():
    if (not f.endswith('ipynb')) and f != '.test':
        if os.path.isfile(f):
            os.remove(f)
        elif os.path.isdir(f):
            shutil.rmtree(f)
        else:
            raise "A file that is not a notebook wasn't deleted"

In [ ]:
import boto3
from urllib.parse import urlparse

s3 = boto3.resource('s3')
bucket = s3.Bucket(urlparse(project.artifact_path).netloc)
bucket.objects.filter(Prefix=urlparse(project.artifact_path).path[1:]).delete()